In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
class TreeNode:
    def __init__(self,data,output):
        #data represents the feature it was split
        #data =None for a leaf node
        self.data=data
        #Output represents the class with current majority at this instance
        self.output=output
        #children of the node is stored as a dictionary with key being the value
        #of the feature upon which the node was split and the corresponding value stores 
        #the treenode
        self.children={}
        #index will be used to assign a unique index value to each node
        self.index=-1
        
    def addchild(self,feature_value,obj):
        self.children[feature_value]=obj

In [3]:
class DecisionTreeClassifier:
    def __init__(self):
        self.root=None #root represents the root of the Decison Tree
    def countunique(self,Y):
        #returns frequency count for each class
        freq_map={}
        for i in Y:
            if i not in freq_map:
                freq_map[i]=1
            else:
                freq_map[i]+=1
        return freq_map
    def entropy(self,Y):
        #returns the entropy
        freq_map=self.countunique(Y)
        total=len(Y)
        Entropy=0
        for i in set(Y):
            p=freq_map[i]/total
            Entropy+=(-p)*math.log2(p)
        return Entropy
    def gainratio(self,X,Y,selected_feature):
        #returns the gain ratio
        info_original=self.entropy(Y);
        info_features=0
        split_info=0
        
        values = set(X[:,selected_feature])
        df=pd.DataFrame(X)
        df[df.shape[1]]=Y
        initialsize=df.shape[0]
        for i in values:
            df1=df[df[selected_feature]==i]
            current_size=df1.shape[0]
            info_features+=current_size/initialsize*self.entropy(df1[df1.shape[1]-1])
            split_info+=(-(current_size/initialsize))*(math.log2(current_size/initialsize))
        #handling for the case when split info will be 0
        if split_info==0:
            return math.inf
        info_gain=info_original-info_features
        gain_ratio=info_gain/split_info
        return gain_ratio
    def Decision_Tree(self,X,Y,features,level,classes):
        #Contains PreOrder Traversal Of Tree
        #if the node consist of only 1 class
        # classes represents the different classes present in the classification problem 
        if len(set(Y))==1:
            print("Level ",level)
            output=None
            for i in classes:
                if i not in Y:
                    print("Count of {} is 0".format(i))
                else:
                    print("Count of {} is {}".format(i,len(Y)))
                    output=i
            print("Current Entropy is 0")
            print("Reached Leaf Node")
            print()
            return TreeNode(None,output)
        #if the no. of features left is 0
        #No feature left to predict upon, we will output the class with the 
        #max xount
        if len(features)==0:
            print("Level ",level)
            freq_map=self.countunique(Y)
            max_count=-math.inf
            output=None
            for i in classes:
                if i not in freq_map:
                    print("Count of {} is 0".format(i))
                else:
                    if freq_map[i] > max_count:
                        max_count=freq_map[i]
                        output=i
                    print("Count of {} is {}".format(i,freq_map[i]))
            print("Current Entropy is ",self.entropy(Y))
            print("Reached Leaf Node")
            print()
            return TreeNode(None, output)
        #Selecting the best feature to split upon
        print("Level ", level)
        max_gain=-math.inf
        final_feature=None
        for f in features:
            current_gain=self.gainratio(X,Y,f)
            if current_gain>max_gain:
                max_gain=current_gain
                final_feature=f
        freq_map=self.countunique(Y)
        output=None
        max_count=-math.inf
        for i in classes:
            if i not in freq_map:
                print("Count of {} is 0".format(i))
            else:
                if freq_map[i] > max_count:
                    max_count=freq_map[i]
                    output=i
                print("Count of {} is {}".format(i,freq_map[i]))
        print("Current Entropy is {}".format(self.entropy(Y)))
        print("Spliiting on feature {} with gain ratio = {}".format(final_feature,max_gain))
        print()
        current_node=TreeNode(final_feature,output)
        unique_values=set(X[:,final_feature])
        df=pd.DataFrame(X)
        df[df.shape[1]]=Y
        index=features.index(final_feature)
        features.remove(final_feature)
        for i in unique_values:
            df1=df[df[final_feature]==i]
            node=self.Decision_Tree(df1.iloc[:,0:df1.shape[1]-1].values,df1.iloc[:,df1.shape[1]-1].values,features,level+1,classes)
            current_node.addchild(i,node)
        features.insert(index,final_feature)
        return current_node
    def fit(self,X,Y):
        #features=[i for i in range(len(X[0]))]
        features = [i for i in range(len(X[0]))]
        classes=set(Y)
        self.root=self.Decision_Tree(X,Y,features,0,classes)
    def predictfor(self,data,node):
        #returns the class for each data point
        
        #we have reached a leaf node
        if len(node.children) == 0 :
            return node.output

        val = data[node.data] 
        # represents the value of feature on which the split was made       
        if val not in node.children :
            return node.output
        
        # Recursively call on the splits
        return self.predictfor(data,node.children[val])
    def predict(self,X):
        Y=np.array([0 for i in range(len(X))])
        for i in range(len(X)):
            Y[i]=self.predictfor(X[i],self.root)
        return Y
    def score(self,X,Y):
        y_pred=self.predict(X)
        count=0
        for i in range(len(Y)):
            if y_pred[i]==Y[i]:
                count+=1
        return count/len(Y)
        

In [4]:
from sklearn import datasets
iris = datasets.load_iris()
df=pd.DataFrame(iris.data)
y=(iris.target)
df.columns = ["sl", "sw", 'pl', 'pw']
def label(val, *boundaries):
    if (val < boundaries[0]):
        return 0
    elif (val < boundaries[1]):
        return 1
    elif (val < boundaries[2]):
        return 2
    else:
        return 3
def toLabel(df, old_feature_name):
    second = df[old_feature_name].mean()
    minimum = df[old_feature_name].min()
    first = (minimum + second)/2
    maximum = df[old_feature_name].max()
    third = (maximum + second)/2
    return df[old_feature_name].apply(label, args= (first, second, third))
df['sl_labeled'] = toLabel(df, 'sl')
df['sw_labeled'] = toLabel(df, 'sw')
df['pl_labeled'] = toLabel(df, 'pl')
df['pw_labeled'] = toLabel(df, 'pw')
df.drop(['sl', 'sw', 'pl', 'pw'], axis = 1, inplace = True)
f=list(df.columns)
tree=DecisionTreeClassifier()
df=df.to_numpy()
tree.fit(df,y)

Level  0
Count of 0 is 50
Count of 1 is 50
Count of 2 is 50
Current Entropy is 1.584962500721156
Spliiting on feature 3 with gain ratio = 0.6996382036222091

Level  1
Count of 0 is 50
Count of 1 is 0
Count of 2 is 0
Current Entropy is 0
Reached Leaf Node

Level  1
Count of 0 is 0
Count of 1 is 10
Count of 2 is 0
Current Entropy is 0
Reached Leaf Node

Level  1
Count of 0 is 0
Count of 1 is 40
Count of 2 is 16
Current Entropy is 0.863120568566631
Spliiting on feature 2 with gain ratio = 0.4334099495621067

Level  2
Count of 0 is 0
Count of 1 is 1
Count of 2 is 0
Current Entropy is 0
Reached Leaf Node

Level  2
Count of 0 is 0
Count of 1 is 39
Count of 2 is 8
Current Entropy is 0.6581912658132185
Spliiting on feature 0 with gain ratio = 0.12674503775809332

Level  3
Count of 0 is 0
Count of 1 is 0
Count of 2 is 1
Current Entropy is 0
Reached Leaf Node

Level  3
Count of 0 is 0
Count of 1 is 14
Count of 2 is 0
Current Entropy is 0
Reached Leaf Node

Level  3
Count of 0 is 0
Count of 1 is 

In [5]:
Y_pred = tree.predict(df)
print("Predictions :",Y_pred)
print()
print("My Score on Training Data",tree.score(df,y)) # Score on training data


Predictions : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 1 2 2 2 1 2 2 1 1 2 2 2 2 2 1 2 2 2 2 1 2 2 2 2 2 2 2 2 2
 2 1]

My Score on Training Data 0.9533333333333334


In [6]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf.fit(iris.data,iris.target)
clf.score(iris.data,iris.target)

1.0